In [ ]:
__author__ = 'Kunal Banerjee'
__date__ = '20240916'
__email__ = 'kunal.banerjee.cse@gmail.com'

## Import libraries

In [ ]:
from __future__ import division
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier, XGBRegressor, plot_importance
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, \
                            mean_squared_error

## Global variables

In [ ]:
PATH_TO_DATA = '../data/'

## Preprocess the data for classification and regression tasks

In [ ]:
def preprocess_data(filename, retainDatetime=False, applyRegressionOnDroppingEdge=False, 
                    updateTarget='equal', num_steps=10):
    print('Prepocessing ', filename)
    data = pd.read_csv(filename, sep=';')
    #print(data.columns)
    
    if retainDatetime == False:
        data = data.drop(['datetime'], axis=1)
        
    data_label = data['anomaly']
    data_label_regr = data_label.copy()
    # Find the indices of the changepoints
    index_cp = data[data['changepoint'] == 1].index
    
    if len(index_cp) != 4 and len(index_cp) != 2:
        print('ERROR:: Unexpected number of changepoints ', len(index_cp), ' detected in ', filename)
        exit()
        
    if len(index_cp) == 2:
        print('Abrupt anomalies detected in ', filename)
    else:
        data_label[index_cp[0]:index_cp[1]] = 0
        data_label[index_cp[2]:index_cp[3]] = 0
        
        if 'equal' == updateTarget:
            for i in range(index_cp[0], index_cp[1]):
                data_label_regr.loc[i] = (i - index_cp[0] + 1)/(index_cp[1] - index_cp[0] + 1)
            if applyRegressionOnDroppingEdge:
                for i in range(index_cp[2], index_cp[3]):
                    # For the dropping edge, the target value for regression should be monotonically decreasing
                    data_label_regr.loc[i] = 1.0 - ((i - index_cp[2] + 1)/(index_cp[3] - index_cp[2] + 1))
        elif 'exponential' == updateTarget:
            for i in range(index_cp[0], index_cp[1]):
                data_label_regr.loc[i] = 1.0/pow(2,(index_cp[1] - i))
            if applyRegressionOnDroppingEdge:
                for i in range(index_cp[2], index_cp[3]):
                    # For the dropping edge, the target value for regression should be monotonically decreasing
                    data_label_regr.loc[i] = 1.0/pow(2,(i - index_cp[2]))
        elif 'step' == updateTarget:
            total_range_ascend = index_cp[1] - index_cp[0]
            incr_value = 1.0/num_steps
            for i in range(index_cp[0], index_cp[1]):
                data_label_regr.loc[i] = ((i - index_cp[0])//(total_range_ascend/num_steps))*incr_value
            if applyRegressionOnDroppingEdge:
                total_range_descend = index_cp[2] - index_cp[1]
                for i in range(index_cp[2], index_cp[3]):
                    # For the dropping edge, the target value for regression should be monotonically decreasing
                    data_label_regr.loc[i] = ((index_cp[3] - i)//(total_range_descend/num_steps))*incr_value
                        
    # Drop the target variables
    data = data.drop(['anomaly', 'changepoint'], axis=1)
    
    return data, data_label, data_label_regr


## Perform classification

In [ ]:
def classification(train, train_label, test, test_label, algorithm='xgb'):
    # create model instance
    if algorithm == 'xgb':
        classifier = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
    elif algorithm == 'svm':
        classifier = SVC(kernel='linear')
    elif algorithm == 'gaussiannb':
        classifier = GaussianNB()
    else:
        print('ERROR:: Unknown classification algorithm: ', algorithm)
        exit()
    # fit model
    classifier.fit(train, train_label)
    # make predictions
    preds = classifier.predict(test)
    return classifier, accuracy_score(test_label, preds), balanced_accuracy_score(test_label, preds), \
           f1_score(test_label, preds), precision_score(test_label, preds), recall_score(test_label, preds)
    

## Perform regression

In [ ]:
def regression(train, train_label, test, test_label, algorithm='xgb'):
    # create model instance
    if algorithm == 'xgb':
        regressor = XGBRegressor(objective ='reg:squarederror', n_estimators = 10, seed = 123)
    elif algorithm == 'svm':
        regressor = SVR(kernel='rbf')
    elif algorithm == 'lasso':
        regressor = Lasso(alpha=0.1)
    elif algorithm == 'ridge':
        regressor = Ridge(alpha=0.1)
    else:
        print('ERROR:: Unknown regression algorithm: ', algorithm)
        exit()
    # fit model
    regressor.fit(train, train_label)
    # make predictions
    preds = regressor.predict(test)
    rmse = mean_squared_error(test_label, preds, squared=True)
    return regressor, rmse
 

## Perform anomaly prediction

In [ ]:
def anomaly_prediction(data, classifier, regressor):
    preds = list()
    for row in data:
        row = row.reshape(1,-1)
        pred = classifier.predict(row)
        if pred != 1:
            pred = regressor.predict(row)
        
        preds.append(pred)
        
    return preds
 

## Evaluate anomaly prediction

In [ ]:
def evaluate_anomaly_prediction(test_label_regr, preds, threshold):
    # Expecting the test label to contain values between 0 and 1, i.e., it's same as test_label_regr
    for i in range(len(test_label_regr)):
        if test_label_regr[i] != 0:
            test_label_regr.loc[i] = 1
    
    # We need to create a copy so that the original preds does not get modified
    predsCopy = list()
    
    for i in range(len(preds)):
        if preds[i] >= threshold:
            predsCopy.append(1)
        else:
            predsCopy.append(0)
            
    return accuracy_score(test_label_regr, predsCopy), balanced_accuracy_score(test_label_regr, predsCopy), \
           f1_score(test_label_regr, predsCopy), precision_score(test_label_regr, predsCopy), \
           recall_score(test_label_regr, predsCopy)


## Code for training and testing anomaly prediction

In [ ]:
def train_and_test():
    # Create the list of files to be used for training
    trainFiles = list()
    for i in range(3):
        filename = PATH_TO_DATA + 'valve2/' + str(i) + '.csv'
        trainFiles.append(filename)
    
    print('Files used for training: ', trainFiles)
    
    # Create the list of files to be used for testing
    testFiles = list()
    for i in range(3,4):
        filename = PATH_TO_DATA + 'valve2/' + str(i) + '.csv'
        testFiles.append(filename)
        
    print('Files used for testing: ', testFiles)
    
    # We don't need to retain datetime for classifiers such as XGBClassifier
    retainDatetime=False
    
    columns = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current', 'Pressure', 'Temperature', 
               'Thermocouple', 'Voltage', 'Volume Flow RateRMS']
    if retainDatetime:
        columns = columns.insert(0, 'datetime')

    train = pd.DataFrame(columns=columns)
    
    train_label = pd.Series(dtype=np.float64)
    train_label_regr = pd.Series(dtype=np.float64)
    for file in trainFiles:
        data, data_label, data_label_regr = preprocess_data(file)
        train = pd.concat([train, data], ignore_index=True)
        train_label = pd.concat([train_label, data_label], ignore_index=True)
        train_label_regr = pd.concat([train_label_regr, data_label_regr], ignore_index=True)
        
    test = pd.DataFrame(columns=columns)
    test_label = pd.Series(dtype=np.float64)
    test_label_regr = pd.Series(dtype=np.float64)
    for file in testFiles:
        data, data_label, data_label_regr = preprocess_data(file)
        test = pd.concat([test, data], ignore_index=True)
        test_label = pd.concat([test_label, data_label], ignore_index=True)
        test_label_regr = pd.concat([test_label_regr, data_label_regr], ignore_index=True)
    
    # Scale the data
    # Note that due to scaling, pandas.DataFrame becomes numpy.ndarray
    scaler = StandardScaler()
    if retainDatetime == False:
        scaler.fit(train)
        train = scaler.transform(train)
        test = scaler.transform(test)
    else:
        cols_to_scale = columns
        cols_to_scale.remove('datetime')
        scaler.fit(train[cols_to_scale])
        train[cols_to_scale] = scaler.transform(train[cols_to_scale])
        test[cols_to_scale] = scaler.transform(test[cols_to_scale])
    
    classifier, accuracy, balanced_accuracy, f1, precision, recall = \
        classification(train, train_label, test, test_label, algorithm='gaussiannb')
    print('Classification:: Accuracy: ', accuracy, ' Balanced Accuracy: ', balanced_accuracy, 
          ' F1: ', f1, ' Precision: ', precision, ' Recall: ', recall)
    
    # Plot feature importance of classifier
    #plot_importance(classifier)
    #plt.show()
    
    regressor, rmse = regression(train, train_label_regr, test, test_label_regr, algorithm='ridge')
    print('Regression:: RMSE: ', rmse)
    
    preds = anomaly_prediction(test, classifier, regressor)
    print('Anomaly Prediction')
    for threshold in np.arange(0.5, 1.0, 0.1):
        APaccuracy, APbalanced_accuracy, APf1, APprecision, APrecall = \
            evaluate_anomaly_prediction(test_label_regr, preds, threshold)
        print('Threshold:: ', threshold, ' Accuracy: ', APaccuracy, ' Balanced Accuracy: ', APbalanced_accuracy, 
              ' F1: ', APf1, ' Precision: ', APprecision, ' Recall: ', APrecall)
    

In [ ]:
train_and_test()

## Pre-processing for stat-based anomaly prediction

In [ ]:
def preprocess_data_stat(filename, applyRegressionOnDroppingEdge=True):
    print('Prepocessing for stat-based prediction', filename)
    data = pd.read_csv(filename, sep=';')
    #print(data.columns)
    
    # Drop datetime column
    data = data.drop(['datetime'], axis=1)
        
    data_label = data['anomaly']
    data_label_regr = data_label.copy()
    # Find the indices of the changepoints
    index_cp = data[data['changepoint'] == 1].index
    
    if len(index_cp) != 4 and len(index_cp) != 2:
        print('ERROR:: Unexpected number of changepoints ', len(index_cp), ' detected in ', filename)
        exit()
        
    if len(index_cp) == 2:
        print('Abrupt anomalies detected in ', filename)
    else:
        data_label[index_cp[0]:index_cp[1]] = 0
        data_label[index_cp[2]:index_cp[3]] = 0
        
        data_label_regr[index_cp[0]:index_cp[1]] = 1     
        if applyRegressionOnDroppingEdge:
            data_label_regr[index_cp[2]:index_cp[3]] = 1
        else:
            data_label_regr[index_cp[2]:index_cp[3]] = 0
           
    # Drop the target variables
    data = data.drop(['anomaly', 'changepoint'], axis=1)
    
    return data, data_label, data_label_regr


## Evaluate stat-based anomaly prediction

In [ ]:
def evaluate_anomaly_prediction_stat(train, train_label, test, test_label):
    train_mean = train.mean()
    train_std = train.std()
    best_f1 = 0.0
    best_num_vars = 0
    best_deviation = 0.0
    # The variable num_vars controls the number of variables that should cross threshold for an anomaly
    for num_vars in range(1, len(train.columns)+1):
        # The variable deviation controls the standard deviation that needs to be crossed for an anomaly
        for deviation in np.arange(1.0, 4.0, 0.1):
            preds = list()
            for index, row in train.iterrows():
                vars_crossing_threshold = 0
                for col in train.columns:
                    if row[col] > (train_mean[col] + deviation * train_std[col]) or \
                       row[col] < (train_mean[col] - deviation * train_std[col]):
                        vars_crossing_threshold += 1
                if vars_crossing_threshold >= num_vars:
                    preds.append(1)
                else:
                    preds.append(0)
            
            current_f1 = f1_score(train_label, preds)
            if current_f1 > best_f1:
                best_f1 = current_f1
                best_num_vars = num_vars
                best_deviation = deviation
    print('Best training F1-score: ', best_f1)
    print('Standard deviation corresponding to best F1-score: ', best_deviation)
    print('Number of variables that should cross the thtreshold for the best F1-score: ', best_num_vars)
                
    preds = list()
    for index, row in test.iterrows():
        vars_crossing_threshold = 0
        for col in test.columns:
            if row[col] > (train_mean[col] + best_deviation * train_std[col]) or \
               row[col] < (train_mean[col] - best_deviation * train_std[col]):
                vars_crossing_threshold += 1
        if vars_crossing_threshold >= best_num_vars:
            preds.append(1)
        else:
            preds.append(0)
    
    # This following line was added because otherwise the test_label_regr values were being read as
    # real numbers leading to error while computing classification metrics
    test_label = test_label.astype(int)
    
    return accuracy_score(test_label, preds), balanced_accuracy_score(test_label, preds), \
           f1_score(test_label, preds), precision_score(test_label, preds), recall_score(test_label, preds)


## Stat-based anomaly prediction

In [ ]:
def train_and_test_stat():
    # Create the list of files to be used for training
    trainFiles = list()
    for i in range(3):
        filename = PATH_TO_DATA + 'valve2/' + str(i) + '.csv'
        trainFiles.append(filename)
    
    print('Files used for training: ', trainFiles)
    
    # Create the list of files to be used for testing
    testFiles = list()
    for i in range(3,4):
        filename = PATH_TO_DATA + 'valve2/' + str(i) + '.csv'
        testFiles.append(filename)
        
    print('Files used for testing: ', testFiles)
    
    # We don't need to retain datetime for stat-based anomaly detection and prediction
    columns = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current', 'Pressure', 'Temperature', 
               'Thermocouple', 'Voltage', 'Volume Flow RateRMS']

    train = pd.DataFrame(columns=columns)
    
    train_label = pd.Series(dtype=np.float64)
    train_label_regr = pd.Series(dtype=np.float64)
    for file in trainFiles:
        data, data_label, data_label_regr = preprocess_data_stat(file)
        train = pd.concat([train, data], ignore_index=True)
        train_label = pd.concat([train_label, data_label], ignore_index=True)
        train_label_regr = pd.concat([train_label_regr, data_label_regr], ignore_index=True)
        
    test = pd.DataFrame(columns=columns)
    test_label = pd.Series(dtype=np.float64)
    test_label_regr = pd.Series(dtype=np.float64)
    for file in testFiles:
        data, data_label, data_label_regr = preprocess_data(file)
        test = pd.concat([test, data], ignore_index=True)
        test_label = pd.concat([test_label, data_label], ignore_index=True)
        test_label_regr = pd.concat([test_label_regr, data_label_regr], ignore_index=True)
    
    accuracy, balanced_accuracy, f1, precision, recall = \
        evaluate_anomaly_prediction_stat(train, train_label, test, test_label)
    print('Anomaly Datection:: Accuracy: ', accuracy, ' Balanced Accuracy: ', balanced_accuracy, 
          ' F1: ', f1, ' Precision: ', precision, ' Recall: ', recall)
    
    accuracy, balanced_accuracy, f1, precision, recall = \
        evaluate_anomaly_prediction_stat(train, train_label_regr, test, test_label_regr)
    print('Anomaly Prediction:: Accuracy: ', accuracy, ' Balanced Accuracy: ', balanced_accuracy, 
          ' F1: ', f1, ' Precision: ', precision, ' Recall: ', recall)
    

In [ ]:
train_and_test_stat()

## Anomaly prediction with shifted anomaly detection

In [ ]:
def train_and_test_shifted():
    # Create the list of files to be used for training
    trainFiles = list()
    for i in range(3):
        filename = PATH_TO_DATA + 'valve2/' + str(i) + '.csv'
        trainFiles.append(filename)
    
    print('Files used for training: ', trainFiles)
    
    # Create the list of files to be used for testing
    testFiles = list()
    for i in range(3,4):
        filename = PATH_TO_DATA + 'valve2/' + str(i) + '.csv'
        testFiles.append(filename)
        
    print('Files used for testing: ', testFiles)
    
    # We don't need to retain datetime for classifiers such as XGBClassifier
    retainDatetime=False
    
    columns = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current', 'Pressure', 'Temperature', 
               'Thermocouple', 'Voltage', 'Volume Flow RateRMS']
    if retainDatetime:
        columns = columns.insert(0, 'datetime')

    train = pd.DataFrame(columns=columns)
    
    train_label = pd.Series(dtype=np.float64)
    train_label_regr = pd.Series(dtype=np.float64)
    for file in trainFiles:
        data, data_label, data_label_regr = preprocess_data_stat(file)
        train = pd.concat([train, data], ignore_index=True)
        train_label = pd.concat([train_label, data_label], ignore_index=True)
        train_label_regr = pd.concat([train_label_regr, data_label_regr], ignore_index=True)
        
    test = pd.DataFrame(columns=columns)
    test_label = pd.Series(dtype=np.float64)
    test_label_regr = pd.Series(dtype=np.float64)
    for file in testFiles:
        data, data_label, data_label_regr = preprocess_data_stat(file)
        test = pd.concat([test, data], ignore_index=True)
        test_label = pd.concat([test_label, data_label], ignore_index=True)
        test_label_regr = pd.concat([test_label_regr, data_label_regr], ignore_index=True)
    
    # Scale the data
    # Note that due to scaling, pandas.DataFrame becomes numpy.ndarray
    scaler = StandardScaler()
    if retainDatetime == False:
        scaler.fit(train)
        train = scaler.transform(train)
        test = scaler.transform(test)
    else:
        cols_to_scale = columns
        cols_to_scale.remove('datetime')
        scaler.fit(train[cols_to_scale])
        train[cols_to_scale] = scaler.transform(train[cols_to_scale])
        test[cols_to_scale] = scaler.transform(test[cols_to_scale])
    
    classifier, accuracy, balanced_accuracy, f1, precision, recall = \
        classification(train, train_label, test, test_label, algorithm='xgb')
    print('Classification:: Accuracy: ', accuracy, ' Balanced Accuracy: ', balanced_accuracy, 
          ' F1: ', f1, ' Precision: ', precision, ' Recall: ', recall)
    

In [ ]:
train_and_test_shifted()